## Implicit collaborative filtering pipeline for gracioustyle

### Background

This is the development pipeline to apply the implicit item-based recommendation system algorithms for gracioustyle customers. The aim is to test and compare a few algorithms including  
* Alternating Least Square (with Spark implemenation)
* Bayesian Personalized Ranking
* Logistic Matrix Factorization
* Nearest Neighbours  
  
Further development phrase will incorporate deep learning method to test if any uplift can be achieved on top of the above methods.

Note this pipeline is currently in development phase. 

### Import libraries

In [2]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import implicit
from implicit.datasets.movielens import get_movielens
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import tqdm
import codecs

### Import data

Order data has been merged with product table to build a master source data in json/csv format.

In [55]:
retail_data = pd.read_json('data/cus_user_quality_date_0716.json') # This may take a couple minutes
# df2 = pd.read_csv('data/product_brand_type_color.csv') # This may take a couple minutes

In [56]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120442 entries, 0 to 120441
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMER_DIM_KEY   120442 non-null  int64  
 1   PRODUCT_DIM_KEY    120442 non-null  int64  
 2   QUANTITY           120441 non-null  float64
 3   PRODUCT_NAME       120435 non-null  object 
 4   COLOR              120442 non-null  object 
 5   PRODUCT_TYPE       120442 non-null  object 
 6   CATEGORY           120442 non-null  object 
 7   BRAND              120442 non-null  object 
 8   RESOLUTION_STATUS  120442 non-null  object 
 9   ORDER_DATE_KEY     120442 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 9.2+ MB


Some data subseting has been done based on the following conditions,  
  
* Only contains the order history on products that have had orders since 2018. This is used as a proxy as the current product catelogue. 
* Resolution status in 'Completed' or 'Pending'.
* De-duplicate the identical rows.

In [57]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]
cleaned_retail = cleaned_retail[cleaned_retail['PRODUCT_DIM_KEY'].isin(cleaned_retail[cleaned_retail['ORDER_DATE_KEY'] > 17533].PRODUCT_DIM_KEY)]
cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)
cleaned_retail = cleaned_retail[cleaned_retail.RESOLUTION_STATUS.isin(['COMPLETED', 'PENDING'])]
print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

Duplicated rows: 280


In [58]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51810 entries, 0 to 120440
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CUSTOMER_DIM_KEY   51810 non-null  int64  
 1   PRODUCT_DIM_KEY    51810 non-null  int64  
 2   QUANTITY           51810 non-null  float64
 3   PRODUCT_NAME       51810 non-null  object 
 4   COLOR              51810 non-null  object 
 5   PRODUCT_TYPE       51810 non-null  object 
 6   CATEGORY           51810 non-null  object 
 7   BRAND              51810 non-null  object 
 8   RESOLUTION_STATUS  51810 non-null  object 
 9   ORDER_DATE_KEY     51810 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.3+ MB


### Threhold function for cut-off

In [59]:
item_lookup = cleaned_retail[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']].drop_duplicates() # Only get unique item/description pairs
#item_lookup['PRODUCT_DIM_KEY'] = item_lookup.PRODUCT_DIM_KEY.astype(str) # Encode as strings for future lookup ease

In [7]:
def threshold_likes(df, uid_min, mid_min, user_column = 'CUSTOMER_DIM_KEY', item_column = 'PRODUCT_DIM_KEY'):
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby(user_column)[item_column].count()
        df = df[~df[user_column].isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby(item_column)[user_column].count()
        df = df[~df[item_column].isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby(user_column)[item_column].count().min() >= mid_min)
    assert(df.groupby(item_column)[user_column].count().min() >= uid_min)
    
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [9]:
df_lim = threshold_likes(cleaned_retail, 2, 2)

Starting likes info
Number of users: 27143
Number of models: 17314
Sparsity: 0.011%
Ending likes info
Number of users: 7088
Number of models: 5368
Sparsity: 0.064%


### User-product matrix function

In [15]:
def create_matrix(cleaned_retail, row_feature, col_feature):
    cleaned_retail[row_feature] = cleaned_retail[row_feature].astype(int) # Convert to int for customer ID
    cleaned_retail = cleaned_retail[[col_feature, 'QUANTITY', row_feature]] # Get rid of unnecessary info
    grouped_cleaned = cleaned_retail.groupby([row_feature, col_feature]).sum().reset_index() # Group together
    grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
    # indicate purchased
    grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
    
    customers = list(np.sort(grouped_purchased[row_feature].unique())) # Get our unique customers
    products = list(grouped_purchased[col_feature].unique()) # Get our unique products that were purchased
    quantity = list(grouped_purchased.QUANTITY) # All of our purchases

    rows = grouped_purchased[row_feature].astype('category', CategoricalDtype(categories = customers)).cat.codes 
    # Get the associated row indices
    cols = grouped_purchased[col_feature].astype('category', CategoricalDtype(categories = products)).cat.codes 
    # Get the associated column indices
    purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
    return purchases_sparse

### Processing data

Further process the data and source relevant information. 

In [16]:
cleaned_retail['CUSTOMER_DIM_KEY'] = cleaned_retail['CUSTOMER_DIM_KEY'].astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['PRODUCT_DIM_KEY', 'QUANTITY', 'CUSTOMER_DIM_KEY']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY']).sum().reset_index() # Group together
grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
customers = list(np.sort(grouped_purchased['CUSTOMER_DIM_KEY'].unique())) # Get our unique customers
products = list(grouped_purchased['PRODUCT_DIM_KEY'].unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.QUANTITY) # All of our purchases
rows = grouped_purchased.CUSTOMER_DIM_KEY.astype('category', CategoricalDtype(categories = customers)).cat.codes 
# Get the associated row indices
cols = grouped_purchased.PRODUCT_DIM_KEY.astype('category', CategoricalDtype(categories = products)).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\david_qi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### Create user-product matrix

In [19]:
purchases_sparse = create_matrix(df_lim, 'CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY')

### Train and test data prep

In [20]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [27]:
#product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)

In [21]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### Function to calculate AUC (evaluation metric)

In [22]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

In [23]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

### Function to calculate AUC in cross-validation

In [24]:
def calc_mean_auc_cv(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [25]:
def calc_mean_auc_pop(training_set, altered_users, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    
    return float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

### Test fit a raw ALS model

In [26]:
model = AlternatingLeastSquares()
ratings = (bm25_weight(product_train, B=0.9) * 5).tocsr()
model.fit(ratings)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

(0.699, 0.731)

ALS without tuning results in 0.699 AUC that's worse then the 'populars'

### Generic traning function

In [16]:
import time
def train_model(ratings, model_name, factors =20, regularization=0.1,alpha = 10):
    # lets weight these models by bm25weight.

    if model_name == "als":
        model = AlternatingLeastSquares(factors =factors, regularization=regularization)
        print("weighting matrix by bm25_weight")
        ratings = (bm25_weight(ratings, B=0.9) * 5).tocsr()
    elif model_name == "bpr":
        model = BayesianPersonalizedRanking()

    elif model_name == "lmf":
        model = LogisticMatrixFactorization()

    elif model_name == "tfidf":
        model = TFIDFRecommender()

    elif model_name == "cosine":
        model = CosineRecommender()

    elif model_name == "bm25":
        model = BM25Recommender(B=0.2)

    else:
        raise NotImplementedError("TODO: model %s" % model_name)
    # train the model
    print("training model %s", model_name)
    start = time.time()
    model.fit(ratings*alpha)
    print("trained model '%s' in %s", model_name, time.time() - start)
        
    item_vecs = model.user_factors
    user_vecs = model.item_factors
    return item_vecs, user_vecs, model

### ALS hyper tuning with cross validation

In [27]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation_als(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9) * 5).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)

    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                model = AlternatingLeastSquares(factors = factor, regularization = regu)
                model.fit(training_set*alpha)
                item_vecs = model.user_factors
                user_vecs = model.item_factors
                store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                print(factor, regu, alpha, store_auc)
                
                if store_auc > best_auc:
                    best_auc = store_auc
                    best_factors = factor
                    best_reg = regu
                    best_alpha = alpha
                    best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best factor: ', best_factors)
    print('Best regularization: ', best_reg)
    print('Best alpha: ', best_alpha)
    return best_auc, pop_auc, best_factors, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [28]:
cross_validation_als(purchases_sparse, 'als', param_grid)

The mean AUC of popularity benchmark:  0.731



10 0.0 1 0.799



10 0.0 10 0.819



10 0.0 50 0.814



10 0.0 100 0.807


...


120 100.0 50 0.816



120 100.0 100 0.798



120 100.0 500 0.785



120 100.0 1000 0.787
Best AUC:  0.832
Pop AUC:  0.731
Best factor:  40
Best regularization:  0.1
Best alpha:  10


(0.832, 0.731, 40, 0.1, 10)

### Fit the best ALS based on CV output

In [32]:
training_set = (bm25_weight(product_train, B=0.9) * 5).tocsr()
best_als_model = AlternatingLeastSquares(factors = 40, regularization = 0.1)
best_als_model.fit(training_set * 10)

In [67]:
item_vecs = best_als_model.user_factors
user_vecs = best_als_model.item_factors

### Make recommendations with ALS

Process full data with BM25 weight distance measurement to be consistent with the training apporach

In [36]:
full_set = (bm25_weight(product_test, B=0.9) * 5).tocsr()

Prepare the customer and product array

In [72]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

Extract the actual orders of a customer from full data

In [75]:
get_items_purchased(31777, full_set, customers_arr, products_arr, item_lookup)

,PRODUCT_DIM_KEY,PRODUCT_NAME
5928,124068,Silk & Lingerie Wash Fragrance-Free 64 oz
7083,124069,Silk & Lingerie Wash Original Floral Fusion 64 oz


Make recommendations based on the model generated user_vecs and item_vecs. Refer to the rec_items function below. Note that, we can control if or not we want to recommend the products that customers have already purchased. At the moment, we have filter out the purchased products. 

In [78]:
rec_items(31777, full_set, user_vecs, item_vecs, customers_arr, products_arr, item_lookup, num_items = 10)

,PRODUCT_DIM_KEY,PRODUCT_NAME
0,132959,"Dryer Sachets Summer Verbena, 2-pack and cleanser"
1,102852,Felt Table Liner Padding 60 x 136 Rectangle
2,102841,Felt Table Liner Padding 70 Round
3,102838,Felt Table Liner Padding 60 Round
4,103740,"Isaphire Rubis Napkin 22""X22"", 100% Cotton"
5,133476,Rectangular Pave w/White Crystals Platinum Fra...
6,124053,Laundry Wash Summer Verbena 64 oz
7,124043,Linen Wash Original Floral Fusion 64 oz
8,509700,Idole De Christofle Sterling Silver Lariat Nec...
9,556814,Rustic Garden White Large Footed Planter W/ Em...


### Test fit a raw LMF model

In [ ]:
# bpr lmf als bm25

item_vecs, user_vecs, model = train_model(product_train, 'lmf')
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

### BPR hyper tuning with cross validation

In [54]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001]
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [0.1],
              'regularization': [0.1],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation_bpr(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = BayesianPersonalizedRanking(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [55]:
cross_validation_bpr(purchases_sparse, 'bpr', param_grid)


10 0.1 0.1 1 0.822 0.742



10 0.1 0.1 10 0.825 0.742



10 0.1 0.1 50 0.82 0.742



10 0.1 0.1 100 0.824 0.742



10 0.1 0.1 500 0.826 0.742



10 0.1 0.1 1000 0.818 0.742


KeyboardInterrupt: 

### LMF hyper tuning with cross validation

In [56]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001],
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001],
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation_lmf(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = LogisticMatrixFactorization(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [57]:
cross_validation_lmf(purchases_sparse, 'lmf', param_grid)


10 0.0 1 1 0.772 0.742


KeyboardInterrupt: 

### Fit ALS with RMSE as the evaluation metric (testing)

In [68]:
def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # train ALS model
            model = AlternatingLeastSquares(
                iterations=num_iters,
                factors=rank,
                regularization=reg,)
            model.fit(train_data)
            # make prediction
            valid_data = validation_data.map(lambda p: (p[0], p[1]))
            predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
            # get the rating result
            ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
            # get the RMSE
            MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
            error = math.sqrt(MSE)
            print('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model

### Function to extract actual purchases

In [69]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items  
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    #print(purchased_ind)
    return item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin(prod_codes)]


In [71]:
item_lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36922 entries, 0 to 120395
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT_DIM_KEY  36922 non-null  int64 
 1   PRODUCT_NAME     36922 non-null  object
dtypes: int64(1), object(1)
memory usage: 865.4+ KB


In [72]:
customers_arr[:10]

array([31766, 31767, 31768, 31769, 31770, 31771, 31772, 31773, 31774,
       31775], dtype=int64)

In [105]:
get_items_purchased(31955, product_train, customers_arr, products_arr, item_lookup)

,PRODUCT_DIM_KEY,PRODUCT_NAME
1810,134335,"Eloise Diamant Tablecloth 69""X120"", 100% Cotto..."
42493,111519,Imperial Yellow/Green Wine Glass 8.25 in High ...


In [106]:
rec_items(31955, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,PRODUCT_DIM_KEY,PRODUCT_NAME
0,99025,Chrysanthemum Sample Swatch
1,152018,Royal 71 x 160 Rect Tablecloth
2,111518,Imperial Yellow/Green Water Glass 8.75 in High...
3,154754,Carlton Custom Cotton Tablecloth
4,102864,Felt Table Liner 78 x 114 Oval Table Liner\n
5,197160,Bello Bath Towel 28x59
6,136939,Polylin 19 x 19 Napkins
7,100295,St Moritz 90 X 90 Full/Queen Blanket
8,138380,Mar a Lago Plum Tray
9,197153,Bello Bath Towel 28x59


### Function to make recommendation based on the model outputs

In [70]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': codes, 'PRODUCT_NAME': descriptions}) # Create a dataframe 
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']] # Switch order of columns around

In [90]:
get_items_purchased(37769, product_train, customers_arr, products_arr, item_lookup)

,PRODUCT_DIM_KEY,PRODUCT_NAME
46051,105046,Renaissance Grenade Tablecloth 93 x 144
49277,121173,Two Block Monogram Hand 14 x 21 Linen Embroide...


In [92]:
rec_items(37769, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,PRODUCT_DIM_KEY,PRODUCT_NAME
0,132979,"Dot Green Wine (Set Of 4) 8 In, 7 Oz"
1,109321,Four Crab Napkin Rings\n
2,136539,Six Cornelia 22 x 30 Kitchen Towels
3,109636,Hommage Five-Piece Place Setting
4,208755,Metallic Silver/Concord Tote
5,132983,"Dot Green Champagne (Set Of 4) 9 In, 5 Oz"
6,172934,Black Orchid Cake Server
7,141359,Old St. Nick Utensil Holder 7 in H x 5 in
8,101697,Avonlea 90 x 144 Rect Tablecloth\n
9,119699,Yellow Bird Needlepoint 16 x 16 Square Pillow\n
